# How to handle cases where no queries are generated

Sometimes, a query analysis technique may allow for any number of queries to be generated - including no queries! In this case, our overall chain will need to inspect the result of the query analysis before deciding whether to call the retriever or not.

We will use mock data for this example.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# LLM

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

texts = ["Harrison worked at Kensho"]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_texts(
    texts,
    embeddings,
)
retriever = vectorstore.as_retriever()

# Query analysis
We will use function calling to structure the output. 

However, we will configure the LLM such that is doesn't NEED to call the function representing a search query (should it decide not to). 

We will also then use a prompt to do query analysis that explicitly lays when it should and shouldn't make a search.

In [16]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of job records."""

    query: str = Field(
        ...,
        description="Similarity search query applied to job record.",
    )

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI


In [17]:
system = """You have the ability to issue search queries to get information to help answer user information.

You do not NEED to look things up. If you don't need to, then just respond normally."""

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

In [19]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.bind_tools([Search])
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [20]:
query_analyzer.invoke("where did Harrison Work")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BpypJ2sIuOCQ29AwzdoHVKkW', 'function': {'arguments': '{"query":"Harrison"}', 'name': 'Search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 95, 'total_tokens': 109}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-13c9af1a-5602-4426-a4b1-b3e4fe645705-0', tool_calls=[{'name': 'Search', 'args': {'query': 'Harrison'}, 'id': 'call_BpypJ2sIuOCQ29AwzdoHVKkW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 14, 'total_tokens': 109})

In [21]:
query_analyzer.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 93, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-9f909a6d-cec4-494a-888d-9cc3f08cdcab-0', usage_metadata={'input_tokens': 93, 'output_tokens': 10, 'total_tokens': 103})

# Retrieval with query analysis

So how would we include this in a chain? Let's look at an example below.



In [22]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.runnables import chain

output_parser = PydanticToolsParser(tools=[Search])

In [23]:
@chain
def custom_chain(question):
    response = query_analyzer.invoke(question)
    if "tool_calls" in response.additional_kwargs:
        query = output_parser.invoke(response)
        docs = retriever.invoke(query[0].query)
        # Could add more logic - like another LLM call - here
        return docs
    else:
        return response

In [24]:
custom_chain.invoke("where did Harrison Work")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='Harrison worked at Kensho'),
 Document(page_content='Harrison worked at Kensho')]

In [25]:
custom_chain.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 93, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9925913-aea0-4fad-9fe8-3af8946fa069-0', usage_metadata={'input_tokens': 93, 'output_tokens': 10, 'total_tokens': 103})